# Импорт необходимых библиотек

In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def morphological_skeleton(binary_image, structuring_element):
    skeleton = np.zeros_like(binary_image)  # Пустое изображение для скелета
    temp = np.zeros_like(binary_image)  # Временное изображение
    eroded = binary_image.copy()  # Начинаем с исходного изображения
    
    while True:
        eroded = cv2.erode(eroded, structuring_element)
        opened = cv2.dilate(eroded, structuring_element)
        temp = cv2.subtract(eroded, opened)
        skeleton = cv2.bitwise_or(skeleton, temp)
        if cv2.countNonZero(eroded) == 0:
            break
    
    return skeleton

In [ ]:

# Функция для построения морфологического скелета
def morphological_skeleton(binary_image, structuring_element):
    skeleton = np.zeros_like(binary_image)  # Пустое изображение для скелета
    temp = np.zeros_like(binary_image)  # Временное изображение
    eroded = binary_image.copy()  # Начинаем с исходного изображения
    
    while True:
        # Морфологическое сужение (erosion)
        eroded = cv2.erode(eroded, structuring_element)
        # Морфологическое расширение (dilation) после erosion
        opened = cv2.dilate(eroded, structuring_element)
        # Разница между текущим eroded и opened
        temp = cv2.subtract(eroded, opened)
        # Добавляем результат в скелет
        skeleton = cv2.bitwise_or(skeleton, temp)
        # Если изображение полностью исчезло, выходим из цикла
        if cv2.countNonZero(eroded) == 0:
            break
    
    return skeleton

# Функция восстановления изображения по морфологическому скелету
def reconstruct_image(skeleton, structuring_element):
    reconstructed = np.zeros_like(skeleton)
    for i in range(skeleton.max()):  # Цикл по итерациям
        dilated = cv2.dilate(skeleton, structuring_element, iterations=i)
        reconstructed = cv2.bitwise_or(reconstructed, dilated)
    return reconstructed

# Загрузка бинарного изображения (чёрно-белое изображение)
image_path = 'binary_image.png'  # Укажите путь к вашему изображению
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Преобразовываем изображение в бинарное (чёрно-белое)
_, binary_image = cv2.threshold(original_image, 127, 255, cv2.THRESH_BINARY)

# Определяем структурирующий элемент
structuring_element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))  # Крестовидный элемент

# Построение морфологического скелета
skeleton = morphological_skeleton(binary_image, structuring_element)

# Восстановление изображения по скелету
reconstructed_image = reconstruct_image(skeleton, structuring_element)

# Отображение результатов
plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.title('Исходное изображение')
plt.imshow(binary_image, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title('Морфологический скелет')
plt.imshow(skeleton, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title('Восстановленное изображение')
plt.imshow(reconstructed_image, cmap='gray')
plt.axis('off')

plt.tight_layout()
plt.show()